In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [109]:
fb = pd.read_csv(r"Y:\departments\research_and_development\baseball_operations\clayton_goodiez\csv\polished_dfs\fastball_ready.csv")
bb = pd.read_csv(r"Y:\departments\research_and_development\baseball_operations\clayton_goodiez\csv\polished_dfs\breakingball_ready.csv")
#os = pd.read_csv(r"Y:\departments\research_and_development\baseball_operations\clayton_goodiez\csv\polished_dfs\offspeed_ready.csv")

In [110]:
#all_pitches = pd.concat([fb, bb, os], ignore_index=True)
all_pitches = pd.concat([fb, bb], ignore_index=True)

In [111]:
pitch_counts = all_pitches.groupby('player_name')['count'].sum().reset_index()
pitch_counts.rename(columns={'count': 'total_count'}, inplace=True)
pitch_counts = pitch_counts[pitch_counts['total_count'] >= 300]

In [112]:
arsenals = pd.merge(all_pitches, pitch_counts, on='player_name', how='left')
arsenals['usage'] = arsenals['count'] / arsenals['total_count']
arsenals['arsenal_score'] = arsenals['stuff_plus'] * arsenals['usage']
arsenal_score = arsenals.groupby('player_name')['arsenal_score'].sum().reset_index()
arsenal_score['arsenal_score'] = arsenal_score['arsenal_score'].astype(int)
arsenal_score.sort_values(by='arsenal_score', ascending=False, inplace=True)
arsenal_score = arsenal_score.reset_index(drop=True)
arsenal_score.index = arsenal_score.index + 1

In [113]:
arsenal_score.head(10)

,player_name,arsenal_score
1,"Helsley, Ryan",198
2,"Clase, Emmanuel",182
3,"Jax, Griffin",177
4,"Bautista, Félix",176
5,"Rasmussen, Drew",169
6,"Fairbanks, Pete",168
7,"Megill, Trevor",165
8,"Pressly, Ryan",164
9,"Jackson, Luke",161
10,"Sewald, Paul",161


In [114]:
# select top 25 rows

top_25_rows = arsenal_score[['player_name', 'arsenal_score']].head(25)

# plotting rankings as a table

fig, ax = plt.subplots(figsize=(5, 6))
ax.axis('tight')
ax.axis('off')
ax.set_title('Top 25 Offspeed Stuff+')
table = ax.table(cellText=top_25_rows.values,
                 colLabels=top_25_rows.columns,
                 loc='center',
                 cellLoc='center',
                 colLoc='center',
                 colWidths=[0.4, 0.2])
plt.tight_layout()

# save fig and close

plt.savefig('arsenal_rankings.png', dpi=300)
plt.close()

In [116]:
print(all_pitches[all_pitches['player_name'] == 'Jax, Griffin'])

     Unnamed: 0   player_name pitch_type  stuff_plus  count
159         287  Jax, Griffin         FF         117    244
657         350  Jax, Griffin         SL         203    576
     Unnamed: 0   player_name pitch_type  stuff_plus  count
159         287  Jax, Griffin         FF         117    244


In [118]:
count = arsenal_score['player_name'].apply(lambda name: len(name.split()) > 2).sum()
print(count)


10
